

https://makersportal.com/blog/2018/9/13/audio-processing-in-python-part-i-sampling-and-the-fast-fourier-transform

(DRAFT)- ToDO: possibly break this file out into a few topic files on Frequency, sampling, aliasing and updated with some exercises

# Introduction

Since the publication of Joseph Fourier’s groundbreaking paper in 1822 [see page 525 in text], the use of the Fourier Series has been widespread in applications of engineering ranging from heat transfer to vibration analysis. And more recently, after the evolution of computation and algorithms, the use of the Fast Fourier Transform (FFT) has also become ubiquitous in applications ranging from acoustic analysis to turbulence research and modeling. The notion that sine and cosine waves can be combined to create complex real-world signals is the basis for most of the digital signals that we observe in technology today. In this tutorial, I will describe the basic process for emulating a sampled signal and then processing that signal using the FFT algorithm in Python. This will allow the user to get started with analysis of acoustic-like signals and understand the fundamentals of the Fast Fourier Transform

# Fourier Series

The Fast Fourier Transform, proposed by Cooley and Tukey in 1965, is an efficient computational algorithm of the Discrete Fourier Transform (DFT). The DFT decomposes a signal into a series of the following form:
![title](img/FFT_Sum.png)







where $x_{m}$ is a point in the signal being analyzed and the $X_{k}$ 
is a specific 'mode' or frequency component. Notice that the frequency component can only go up to the length of the signal $(M-1)$, and we will discuss a little later the limitations from there as well (Nyquist).

From above, the complex exponential can be rewritten as sine and cosine functions using the Euler formula:

![title](img/FFT_euler.png)

Such that our series contains sinusoidal waves:

![title](img/FFT_xk_sum.png)

for
k=0, . . .,M-1


We can now see how a signal can be transformed into a series of sinusoidal waves.

# Sine Wave Sampling

The easiest way to test an FFT in Python is to either measure a sinusoidal wave at a known frequency using a microphone, or create a sinusoidal function in Python. Since this section focuses on understanding the FFT, I will demonstrate how to emulate a sampled sine wave using Python. Below is the creation of a sine wave in Python using sampling criteria that emulates a real signal:

In [ ]:
# sampling a sine wave programmatically
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# sampling information
Fs = 44100 # sample rate
T = 1/Fs # sampling period
t = 0.1 # seconds of sampling
N = Fs*t # total points in signal

# signal information
freq = 100 # in hertz, the desired natural frequency
omega = 2*np.pi*freq # angular frequency for sine waves

t_vec = np.arange(N)*T # time vector for plotting
y = np.sin(omega*t_vec)

plt.plot(t_vec,y)
plt.show()

The code above ‘samples’ a sine wave at 44.1 kHz for 0.1 seconds (100 ms). I used a 100 Hz sine wave, so we expect:
![title](img/FFT_cycles.png)

This means that we will get 10 cycles from the 100 Hz sine wave in 0.1 seconds. This also means that we will have 4410 samples for the 10 cycles, or 441 samples per cycle - which is quite a bit for replication of the signal.

# Nyquist Frequency

In digital signal processing:

"In order to recover all Fourier components of a periodic waveform, it is necessary to use a sampling rate fs at least twice the highest waveform frequency"

The above statement requires the user to sample a signal at twice the highest natural frequency of the expected system, or mathematically:

![title](img/FFT_nyq1.png)

Therefore, in the FFT function, the limitation of the frequency component is set by the sample rate, which is typically a little higher than twice the highest natural frequency expected in the system. In the case of acoustics, the sample rates are set at approximately twice the highest frequency that humans are capable of discerning (20 kHz), so the sample rate for audio is at minimum 40 kHz. We often see 44.1 kHz or 48 kHz, which means audio is often sampled correctly above the Nyquist frequency set by the range of the human ear. Therefore, in practice, it is essential to adhere to the following inequality:
![title](img/FFT_nyq2.png)

As a visualization tool, below I have plotted several sampled signals that are sampled around the Nyquist frequency for a 100 Hz sine wave. According to the statement above, if a 100 Hz sine wave is the largest frequency in the system, we should be sampling above 200 Hz.
![title](img/Nyquist_Sampling_example.png)

Test out the affects of aliasing by changing the sampling rates below for Fs, Fs1, and Fs2. 

In [ ]:
# sampling information
Fs = 2000 # sample rate
T = 1/Fs # sampling period
t = 0.04 # seconds of sampling
N = Fs*t # total points in signal

# signal information
freq = 100 # in hertz, the desired natural frequency
omega = 2*np.pi*freq # angular frequency for sine waves

t_vec = np.arange(N)*T # time vector for plotting
y = np.sin(omega*t_vec)

Fs1 = 400 # sample rate
T1 = 1/Fs1 # sampling period
N1 = Fs1*t # total points in signal
t_1=np.arange(N1)*T1
y1 = np.sin(omega*t_1)

Fs2 = 300 # sample rate
T2 = 1/Fs2 # sampling period
N2 = Fs2*t # total points in signal
t_2=np.arange(N2)*T2
y2 = np.sin(omega*t_2)

plt.plot(t_vec,y,t_1,y1,t_2,y2)
plt.show()

The phenomena above, when sampling under the Nyquist frequency is called aliasing. Aliasing can obscure measurements and introduce false peaks in data that can result in inaccurate results. This is why we must sample above the highest natural frequency of the system. Of course, some situations do not warrant pre-determined knowledge of the system, but in those cases methods such as time domain filtering can account for such unexpected behavior. Fortunately, in the field of acoustics, we often don’t need to worry about high frequencies above the typical human hearing range (an exception, of course, is in the ultrasonic range). 


From here, we can investigate the Fast Fourier Transform (FFT) in Python by using our test signal above and the FFT function in Python


# Fast Fourier Transform

This section is informative for two reasons:

we can verify that the sine wave above is sampled correctly

we can gain confidence in our FFT usage by inputting and analyzing a known signal

Now, if we use the example above we can compute the FFT of the signal and investigate the frequency content with an expectation of the behavior outlined above. The Python FFT function in Python is used as follows:

In [ ]:
np.fft.fft(y);

However, it is important to note that the FFT does not produce an immediate physical significance. So we need to divide by the length of the signal, and only take half of the data (single-sided spectrum - not discussed here). From there we need to take the absolute value of the signal to ensure that no imaginary (complex, non-physical) values are present. 

The full FFT algorithm and frequency spectrum plot is shown below:

In [ ]:
# fourier transform and frequency domain
#
#Add some noise
noise = np. random. normal(0, .2, y.shape)
signal = y+noise

Y_k = np.fft.fft(signal)[0:int(N/2)]/N # FFT function from numpy
Y_k[1:] = 2*Y_k[1:] # need to take the single-sided spectrum only
Pxx = np.abs(Y_k) # be sure to get rid of imaginary part

f = Fs*np.arange((N/2))/N; # frequency vector

# plotting
fig,ax = plt.subplots()
plt.plot(f,Pxx,linewidth=5)
plt.ylabel('Amplitude')
plt.xlabel('Frequency [Hz]')
plt.show()

The code takes the FFT of an input signal y (in our case, the sine wave above), which has a length N. It also computes the frequency vector using the number of points and the sampling frequency.

If we were to analyze the frequency and amplitude at the peak of the spectrum plot above (sometimes called a periodogram), we could conclude that the peak is 1 and the frequency is 100 Hz. This returns the amplitude and frequency of our inputted sine wave. Also note the introduction of noise into the signal. The noise is considered an artifact of the computation and is near to zero, so we can neglect it (its amplitude is between 0 and 0.1, so this is a fair assumption). The prediction in this case isn’t particularly impressive, as we could plainly see that the time series above produced a single sine wave at 100 Hz. Below I introduce a more complex signal with three sine waves and some Gaussian noise:

In [ ]:
# fourier transform and frequency domain
#
#Add some noise
noise = np. random. normal(0, .5, y.shape)
signal = 3*y+1.5*np.sin(2*np.pi*155*t_vec)+5.2*np.sin(2*np.pi*283*t_vec)+noise
plt.plot(t_vec,signal)
plt.show()

Y_k = np.fft.fft(signal)[0:int(N/2)]/N # FFT function from numpy
Y_k[1:] = 2*Y_k[1:] # need to take the single-sided spectrum only
Pxx = np.abs(Y_k) # be sure to get rid of imaginary part

f = Fs*np.arange((N/2))/N; # frequency vector

# plotting
fig,ax = plt.subplots()
plt.plot(f,Pxx,linewidth=5)
plt.ylabel('Amplitude')
plt.xlabel('Frequency [Hz]')
plt.show()

It may or may not be obvious to the viewer, but the time series above cannot easily be decomposed into any specific frequency. However, after taking the FFT of the signal, we can easily see there are three resolvable peaks. The noise may have obscured the lowest amplitude signal (around the 150 Hz range), and this is normal for noisy signals. We could conclude, without knowing the original sine wave frequencies or amplitudes, that we had three signals:

100 Hz signal at an amplitude of 3

150 Hz signal at an amplitude of 1.5

280 Hz signal at an amplitude of 4.5

The true inputs were: 100 Hz at an amplitude of 3, 155 Hz at an amplitude of 2, 283 Hz at an amplitude of 5.2, and Gaussian noise at an amplitude of 1. Notice the error associated with the FFT upon introduction of noise. This is important to keep in mind when analyzing signals using FFTs. One way to reduce the error is to record the signal for longer or  try to get the recording device closer to the source (or increase the amplitude of the signal). Occasionally, neither of these methods are possible, which is when other techniques need to be employed such as windowing or time/frequency filtering. I will not cover those more complex signal processing methods here, but if the user is interested in learning about windowing or time/frequency filters, please see the following references: here, here, and here. 

It may or may not be obvious to the viewer, but the time series above cannot easily be decomposed into any specific frequency. However, after taking the FFT of the signal, we can easily see there are three resolvable peaks. The noise may have obscured the lowest amplitude signal (around the 150 Hz range), and this is normal for noisy signals. We could conclude, without knowing the original sine wave frequencies or amplitudes, that we had three signals:

100 Hz signal at an amplitude of 3

150 Hz signal at an amplitude of 1.5

280 Hz signal at an amplitude of 4.5

# Conclusion

The true inputs were: 100 Hz at an amplitude of 3, 155 Hz at an amplitude of 2, 283 Hz at an amplitude of 5.2, and Gaussian noise at an amplitude of 1. Notice the error associated with the FFT upon introduction of noise. This is important to keep in mind when analyzing signals using FFTs. One way to reduce the error is to record the signal for longer or  try to get the recording device closer to the source (or increase the amplitude of the signal). Occasionally, neither of these methods are possible, which is when other techniques need to be employed such as windowing or time/frequency filtering. 